In [1]:
!pip -q install kagglehub
!pip -q install langchain
!pip -q install python-dotenv
!pip -q install streamlit
!pip -q install faiss-cpu
!pip -q install mysql-connector-python
!pip -q install pymysql
!pip -q install sentence-transformers
!pip -q install chromadb
!pip -q install tiktoken
!pip -q install protobuf

In [1]:
from huggingface_hub import login
access_token_read= ""
access_token_write=""
login(token=access_token_read)


In [11]:
!pip -q install google-generativeai


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-google-genai 2.1.10 requires google-ai-generativelanguage<0.7.0,>=0.6.18, but you have google-ai-generativelanguage 0.6.15 which is incompatible.


In [37]:
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI

# 1. Configure Gemini
# genai.configure(api_key="")

# 2. Wrap Gemini inside LangChain
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key="YOUR_API_KEY"
)
llm.invoke("what is AI? ")

AIMessage(content='**Artificial Intelligence (AI)** is a broad field of computer science focused on creating machines that can perform tasks that typically require human intelligence.\n\nIn simpler terms, AI aims to make computers "think" and "learn" like humans do, enabling them to solve problems, make decisions, understand language, recognize patterns, and even perceive the environment.\n\nHere\'s a breakdown of what that means:\n\n1.  **Simulating Human Intelligence:** AI systems are designed to mimic cognitive functions associated with the human mind, such as:\n    *   **Learning:** Acquiring information and rules for using the information.\n    *   **Reasoning:** Using rules to reach approximate or definite conclusions.\n    *   **Problem-solving:** Finding solutions to complex challenges.\n    *   **Perception:** Understanding and interpreting sensory input (like images or sounds).\n    *   **Language Understanding:** Processing and generating human language.\n    *   **Decision-

In [21]:
from langchain.utilities import SQLDatabase

db_user= "root"
db_password= "708sharyar"
db_host= "localhost"
db_name= "sqlbot"

db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}", sample_rows_in_table_info=3)
print(db.table_info)


CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount
1	1	10.00
2	2	15.00
3	3	20.00
*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock

In [14]:
!pip install -q langchain-experimental
!pip install -q langchain-google-genai

In [46]:
from langchain_experimental.sql import SQLDatabaseChain
# from langchain.llms import llm
# db_chain= SQLDatabaseChain.from_llm(llm, db, verbose=True)

db_chain = SQLDatabaseChain.from_llm(
    llm,
    db,
    verbose=True,
    return_direct=True
)

qns1=db_chain.run("How many t-shirt do we have left for nike in extra small size and white color")
print(qns1)



> Entering new SQLDatabaseChain chain...
How many t-shirt do we have left for nike in extra small size and white color
SQLQuery:SQLQuery: SELECT `stock_quantity` FROM `t_shirts` WHERE `brand` = 'Nike' AND `size` = 'XS' AND `color` = 'White' LIMIT 5
SQLResult: [(86,)]
> Finished chain.
[(86,)]


In [47]:
qns1

'[(86,)]'

In [48]:
qns2 = db_chain.run("How many t-shirt do we have left for nike in small size and black color")
print(qns2)




> Entering new SQLDatabaseChain chain...
How many t-shirt do we have left for nike in small size and black color
SQLQuery:SQLQuery: SELECT `stock_quantity` FROM `t_shirts` WHERE `brand` = 'Nike' AND `size` = 'S' AND `color` = 'Black' LIMIT 5
SQLResult: [(67,)]
> Finished chain.
[(67,)]


In [49]:
qns2

'[(67,)]'

In [50]:
qns3=db_chain.run("How much is the price of the inventory for all small size t-shirts?")
print(qns3)
qns3



> Entering new SQLDatabaseChain chain...
How much is the price of the inventory for all small size t-shirts?
SQLQuery:SQLQuery: SELECT SUM(`price` * `stock_quantity`) AS `total_inventory_price` FROM `t_shirts` WHERE `size` = 'S'
SQLResult: [(Decimal('23358'),)]
> Finished chain.
[(Decimal('23358'),)]


"[(Decimal('23358'),)]"

In [51]:
#No need for few shot learning as model is updated I verified
qns4= db_chain.run("SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'")
qns2



> Entering new SQLDatabaseChain chain...
SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'
SQLQuery:SQLQuery: SELECT SUM(`price` * `stock_quantity`) FROM `t_shirts` WHERE `size` = 'S'
SQLResult: [(Decimal('23358'),)]
> Finished chain.


'[(67,)]'

In [52]:
#complex query

qns5= db_chain.run("If we have to sell all the Levi's T-shirts today with discounts applied. How much revenue will my store generate (post discounts)?")
qns5




> Entering new SQLDatabaseChain chain...
If we have to sell all the Levi's T-shirts today with discounts applied. How much revenue will my store generate (post discounts)?
SQLQuery:SQLQuery: SELECT SUM(T1.`stock_quantity` * T1.`price` * (1 - T2.`pct_discount` / 100)) FROM t_shirts AS T1 INNER JOIN discounts AS T2 ON T1.`t_shirt_id` = T2.`t_shirt_id` WHERE T1.`brand` = 'Levi'
SQLResult: [(Decimal('1607.200000'),)]
> Finished chain.


"[(Decimal('1607.200000'),)]"

Few shots learning, more like training model.
Now the thing is that I'm using a really intelligent gemini model, so It is already pretrained, but I'll still train it just because I'm Curious

In [54]:
#Syntax
few_shots= [
    {
       'Question': "",
        'SQLQuery': "",
        'SQLResult': "",
        'Answer': "",
    }
]
#What we do next is that we store these query as embeddings, in some vector DB, we can use HuggingFaceEmbeddings

In [58]:
qns6=db_chain.run("how much sales amount will be generated if we sell all large size t shirts today in nike brand after discounts?")

qns6



> Entering new SQLDatabaseChain chain...
how much sales amount will be generated if we sell all large size t shirts today in nike brand after discounts?
SQLQuery:SQLQuery: SELECT SUM((T1.`price` * (1 - T2.`pct_discount` / 100)) * T1.`stock_quantity`) AS `total_sales_amount` FROM `t_shirts` AS T1 INNER JOIN `discounts` AS T2 ON T1.`t_shirt_id` = T2.`t_shirt_id` WHERE T1.`brand` = 'Nike' AND T1.`size` = 'L'
SQLResult: [(None,)]
> Finished chain.


'[(None,)]'

Making it more Intelligent

In [55]:
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt

print(_mysql_prompt)

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of

In [57]:
from langchain.prompts.prompt import PromptTemplate

example_Prompt= PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult", "Answer"],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\n Answer: {Answer}",
)